In [1]:
# importing libraries
# %pip install mediapipe opencv-python matplotlib
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### Capturing the expected poses in a numpy array

In [2]:
# setting workout timer to 30 seconds
timeout = 20
# initializing the Landmarks array
correct_landmarks = np.array([])

In [3]:
workout_time = time.time() + timeout + 10   # 30 seconds from now (10 seconds for the user to get ready)
cap = cv2.VideoCapture(0)
#recording video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
record = cv2.VideoWriter('expected.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)
# setting up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened() and time.time() < workout_time:
        ret, frame = cap.read()
        if not ret:
            print("Video Ended")
            break
        # detect pose landmarks and render them on the image
        # convert the image from BGR to RGB (opneCV uses BGR by default)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # make detection from pose instance
        results = pose.process(image)
        correct_landmarks = np.append(correct_landmarks, results.pose_landmarks)
        # print(results.pose_landmarks)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # print(results)
        # render pose landmarks on the image
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(51,255,51), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(51,153,255), thickness=2, circle_radius=2))
        
        # cv2.imshow('MediaPipe Pose', frame)
        cv2.imshow('MediaPipe Pose', image)
        # recording video
        record.write(image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    record.release()
    cv2.destroyAllWindows() 
    print("Correct pose recorded successfully!")

Correct pose recorded successfully!


In [4]:
# correct_landmarks
np.save('correct_landmarks.npy', correct_landmarks)
# loading the correct landmarks
saved_landmarks = np.load('correct_landmarks.npy', allow_pickle=True)
print(f"shape of correct_landmarks: {correct_landmarks.shape}")
#printing few landmarks:
for i in range(3):
    print(f"Pose {i}")
    for j in range(5):
        if correct_landmarks[i] is not None:
            print(correct_landmarks[i].landmark[j])

shape of correct_landmarks: (229,)
Pose 0
x: 0.5051068067550659
y: 0.2619391679763794
z: -1.6423507928848267
visibility: 0.9995908141136169

x: 0.5406193733215332
y: 0.19901643693447113
z: -1.6303415298461914
visibility: 0.9981491565704346

x: 0.5601483583450317
y: 0.2009599208831787
z: -1.6302598714828491
visibility: 0.9986960291862488

x: 0.5782780647277832
y: 0.2027454972267151
z: -1.6302030086517334
visibility: 0.9971244931221008

x: 0.4810393750667572
y: 0.1843211054801941
z: -1.6263576745986938
visibility: 0.9986345171928406

Pose 1
x: 0.5048967599868774
y: 0.2617865800857544
z: -1.615944504737854
visibility: 0.9995816349983215

x: 0.5406160950660706
y: 0.1979180872440338
z: -1.596559762954712
visibility: 0.9981552958488464

x: 0.5599924325942993
y: 0.1996561884880066
z: -1.5965086221694946
visibility: 0.9986988306045532

x: 0.5776458382606506
y: 0.20129436254501343
z: -1.5963817834854126
visibility: 0.9971731305122375

x: 0.4797060489654541
y: 0.18422898650169373
z: -1.597539782

In [8]:
# workout_time = time.time() + timeout 
# correct_landmarks = pd.read_csv('correct_landmarks.csv').to_numpy()  
count = 0
poses_matched = 0
cap = cv2.VideoCapture(0)
#recording video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
record = cv2.VideoWriter('actual.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Workout Ended!")
            break
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # make detection from pose instance
        current_results = pose.process(image)
        # print(results.pose_landmarks)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # getting the correct pose landmarks
        correct_pose_landmark = correct_landmarks[count % saved_landmarks.shape[0]]
        count += 1
        #compare the results
        match_count = 0
        color = (0, 0, 255)
        text = "No Pose"
        if(correct_pose_landmark is not None):
            for (act, expected) in zip(current_results.pose_landmarks.landmark, correct_pose_landmark.landmark):
                # calculate the distance between the actual and expected landmarks
                dist = np.sqrt((act.x - expected.x)**2 + (act.y - expected.y)**2 + (act.z - expected.z)**2)
                if dist < 0.25:
                    match_count += 1
            # calculate the percentage of matched landmarks
            match_percent = (match_count / len(correct_pose_landmark.landmark)) * 100
            text = f"{round(match_percent, 2)}% match - "  
            # render pose landmarks on the image
            if match_percent > 75:
                text += "Correct Pose"
                color = (0, 255, 0)
                poses_matched += 1
            else:
                text+= "Incorrect Pose"
                color = (0, 0, 255)
                mp_drawing.draw_landmarks(image, correct_pose_landmark, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(51,255,51), thickness=2, circle_radius=2),
                        mp_drawing.DrawingSpec(color=(51,153,255), thickness=2, circle_radius=2))    
        else:
            print("No Pose Detected!")
            
        mp_drawing.draw_landmarks(image, current_results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))
        cv2.putText(image,text, (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('MediaPipe Pose', image)
        record.write(image)
        
        # render the results from previous cell
        sentence = "No pose detected"
        if cv2.waitKey(10) & 0xFF == ord('q') :
            break
    cap.release()
    record.release()
    cv2.destroyAllWindows() 

### Checking Output Results: 

In [9]:
# print the results
print(f"Total poses matched: {poses_matched}")
print(f"Total poses attempted: {count}")
print(f"Accuracy: {round(poses_matched/count * 100, 2)}%")

Total poses matched: 202
Total poses attempted: 320
Accuracy: 63.12%
